In [26]:
import os
import numpy as np 
import pandas as pd
import geopandas as gpd
import pandana as pdna
import shapely
from shapely.wkt import loads
import shapely.vectorized as sv 
from scipy.spatial.distance import cdist 

In [27]:
abs_path = os.path.dirname(os.path.abspath(__name__))

In [28]:
############# CHANGE HERE ############# 
### Input files and the interested area zip code
network_nodes = abs_path + '/alameda_nodes.csv'
network_edges = abs_path + '/alameda_links.csv'
parcel_data = abs_path + '/Parcels.csv'
zip_code = 94501 # The zip code of Alameda Island is 94501. The user can also use other methods to extract the parcel data of the interested areas
evcuation_node = 734 # Node_id of the evacuation point (it is assumed that all residents evacuate to 1 evacuation point)

############# NO CHANGE HERE #############
# Nodes data processing
# Load the nodes file and covert it to the geopandas dataframe
# This example uses the road network in the Alameda Island with OSMnx
nodes_df = pd.read_csv(network_nodes)
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=[shapely.geometry.Point(xy) for xy in zip(nodes_df.lon, nodes_df.lat)], crs='epsg:4326')
# Load the edges file
edges_df = pd.read_csv(network_edges)

# Parcel data processing
# Load the parcel data
# This example uses the parcel data in the Alameda county (https://data.acgov.org/datasets/2b026350b5dd40b18ed7a321fdcdba81_0/explore)
parcels = pd.read_csv(parcel_data)
# Extract parcel data of the interested area and fill up the missing values if any
parcels = parcels[parcels['SitusZip'] == zip_code]
parcels = parcels.fillna(method = 'ffill') # Fill the missing values by forward propagation
parcels['PARCEL'] = parcels['PARCEL'].astype(int)

/var/folders/p5/yl_qhkks45xd154yl6qd7th40000gn/T/ipykernel_18923/285596952.py:21: DtypeWarning: Columns (3,5,7,8,19) have mixed types. Specify dtype option on import or set low_memory=False.
  parcels = pd.read_csv(parcel_data)


# Find the node_id of the closest node to each parcel

In [29]:
# In this example, all parcels are treated uniformly, with the simple assumption that one car will be used for evacuation per parcel. 
# However, users can change the assumption using the UseCode (https://propinfo.acgov.org/UseCodeList) or other available information.

# Covert the pandas dataframe to the geopandas dataframe
household_parcels = parcels.copy()

household_parcels = gpd.GeoDataFrame(household_parcels, crs='epsg:4326',
                                     geometry=gpd.points_from_xy(household_parcels.xcoord, household_parcels.ycoord))
household_parcels['centroid'] = household_parcels['geometry']
household_parcels = household_parcels.set_geometry('centroid')

household_parcels['c_x'] = household_parcels['xcoord']
household_parcels['c_y'] = household_parcels['ycoord']

# Function to get the closest node to each parcel
def get_closest_node(parcel_x, parcel_y):
    return nodes_id[cdist([(parcel_x, parcel_y)], nodes_xy).argmin()]

# Find the closest node to each parcel
nodes_xy = np.array([nodes_gdf['geometry'].x.values, nodes_gdf['geometry'].y.values]).transpose()
nodes_id = nodes_gdf['node_id'].values

household_parcels['closest_node'] = household_parcels.apply(lambda x: get_closest_node(x['c_x'], x['c_y']), axis=1)

# Create the od pandas dataframe

In [30]:
# Create the od dataframe
od_df = household_parcels[['PARCEL', 'closest_node']].copy().reset_index(drop=True)

# Set the agent ids
od_df['agent_id'] = range(len(od_df)) 

# Set the closest node as the origin
od_df['origin_nid'] = od_df['closest_node']
od_df = od_df.drop('PARCEL', 1)
od_df = od_df.drop('closest_node', 1)

# Set the evacuation node as the destination
od_df['destin_nid'] = evcuation_node

# Set the departure time (it is assumed that all local resident evacuate at the same time)
od_df['hour'] = 0
od_df['quarter'] = 0

/var/folders/p5/yl_qhkks45xd154yl6qd7th40000gn/T/ipykernel_18923/3445467427.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  od_df = od_df.drop('PARCEL', 1)
/var/folders/p5/yl_qhkks45xd154yl6qd7th40000gn/T/ipykernel_18923/3445467427.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  od_df = od_df.drop('closest_node', 1)


# Filter out the trips that do not have routes (Optional)

In [31]:
# pandana network data pre-processing
# Set node_id as the index
nodes_df.index = nodes_df['node_id']

# Convert the data types to the required data types
nodes_df['lon'] = nodes_df['lon'].astype('float64')
nodes_df['lat'] = nodes_df['lat'].astype('float64')
edges_df['start_nid'] = edges_df['start_nid'].astype('int64')
edges_df['end_nid'] = edges_df['end_nid'].astype('int64')
edges_df['length'] = edges_df['length'].astype('float64')
od_df['origin_nid'] = od_df['origin_nid'].astype('int64')
od_df['destin_nid'] = od_df['origin_nid'].astype('int64')

# Create a pandana network
net = pdna.Network(nodes_df['lon'], nodes_df['lat'], edges_df['start_nid'], edges_df['end_nid'], edges_df[['length']], twoway=False)

Generating contraction hierarchies with 10 threads.
Setting CH node vector of size 2083
Setting CH edge vector of size 5460
Range graph removed 5010 edges of 10920
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

In [32]:
# Check if there is a trip between od for each agent
od_df['has_path'] = 0
for i in range(len(od_df)):
    if len(net.shortest_path(od_df['origin_nid'].iloc[i], od_df['destin_nid'].iloc[i])) == 0:
        od_df['has_path'].iloc[i] = 0
    else:
        od_df['has_path'].iloc[i] = 1

In [33]:
# Filter out the trips that do not have routes between the origin and destination
od_df = od_df[od_df['has_path'] == 1]

# Clean up the dataframe
od_df = od_df[['agent_id', 'origin_nid', 'destin_nid', 'hour', 'quarter']]

In [34]:
# Save the dataframe to csv file
od_df.to_csv('od.csv', index=False)